# Data and Model Exploration

This dataframe explores the dataset collected and tests different models to see which model may be best suited for predicting presence with the recorded scenario.

In [1]:
# set working directoy to root directory of the project
if not "working_directory_corrected" in vars():
    %cd ..
    %cd ..
    %cd ..
    working_directory_corrected = True
    
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics

c:\Users\frank\Documents\Research\projects\HomePresencePrediction\environment\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\frank\Documents\Research\projects\HomePresencePrediction\examples\data_recording
c:\Users\frank\Documents\Research\projects\HomePresencePrediction\examples
c:\Users\frank\Documents\Research\projects\HomePresencePrediction


## Data Exploration

Below we load the dataset and observe it's general properties:

In [ ]:
data_frame = pd.read_pickle("examples/simple_scenario/data/recording.pickle")
nr_rooms = (len(data_frame.columns) - 3) // 2

print(f"The dataframe has {len(data_frame)} samples.")
print(f"The example scenario has {nr_rooms} rooms.")
print("below are five random samples")
data_frame.sample(5)

The dataframe has 259200 samples.
The example scenario has 11 rooms.
below are five random samples


,date,weekday,hour,minute,room_0,prediction_0,room_1,prediction_1,room_2,prediction_2,...,room_6,prediction_6,room_7,prediction_7,room_8,prediction_8,room_9,prediction_9,room_10,prediction_10
100300,2020-08-09 15:41:00,6,15,41,0,None,0,None,0,None,...,0,None,0,None,1,None,0,None,1,None
207256,2020-10-22 22:17:00,3,22,17,0,None,0,None,1,None,...,0,None,0,None,0,None,0,None,0,None
249933,2020-11-21 13:34:00,5,13,34,0,None,0,None,0,None,...,0,None,0,None,0,None,0,None,1,None
45709,2020-07-02 17:50:00,3,17,50,0,None,0,None,0,None,...,0,None,0,None,1,None,0,None,0,None
126989,2020-08-28 04:30:00,4,4,30,0,None,0,None,1,None,...,0,None,0,None,0,None,0,None,0,None


Below we calculate the utilization per room. This will tell us how often a presence is detected in the respective rooms in our data.

In [3]:
nr_samples = len(data_frame)
for room_index in range(nr_rooms):
    column = "room_" + str(room_index)
    nr_detections = (data_frame[column].values == 1).sum()
    utilization = int(100* round(nr_detections / nr_samples,2))
    print(f"Room room_{room_index} utilization: {utilization}%")

Room room_0 utilization: 0%
Room room_1 utilization: 0%
Room room_2 utilization: 45%
Room room_3 utilization: 49%
Room room_4 utilization: 44%
Room room_5 utilization: 0%
Room room_6 utilization: 0%
Room room_7 utilization: 4%
Room room_8 utilization: 21%
Room room_9 utilization: 16%
Room room_10 utilization: 42%


## Model Training

Here we test several machine learning models for their ability to learn this data.

Before we test, we split the data into training and test sets

In [4]:
training_features = ["weekday","hour", "minute"]
predicted_features = []
for index in range(nr_rooms):
    predicted_features.append("room_" + str(index))

X = data_frame[training_features]
y = data_frame[predicted_features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"Number Training Samples: {len(X_train)}")
print(f"Number Test Samples: {len(X_test)}")



Number Training Samples: 207360
Number Test Samples: 51840


In [5]:
from pandas import DataFrame


model = tree.DecisionTreeClassifier()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)



In [6]:
print("Accuracy:",metrics.accuracy_score(y_pred=y_pred, y_true=y_test))
predictions_per_room = y_pred.transpose()
for index in range(room_index):
    accuracy = metrics.accuracy_score(y_pred = predictions_per_room[index], y_true = y_test["room_" + str(index)])
    print(f"- Room {index} Accuracy: {accuracy}")


Accuracy: 0.666724537037037
- Room 0 Accuracy: 1.0
- Room 1 Accuracy: 1.0
- Room 2 Accuracy: 0.9226273148148149
- Room 3 Accuracy: 0.9466628086419753
- Room 4 Accuracy: 0.936863425925926
- Room 5 Accuracy: 1.0
- Room 6 Accuracy: 1.0
- Room 7 Accuracy: 1.0
- Room 8 Accuracy: 0.8326967592592592
- Room 9 Accuracy: 0.8403163580246914


In [7]:
model.predict([[6,22,59]])

c:\Users\frank\Documents\Research\projects\HomePresencePrediction\environment\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0]])